In [3]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://animehay.in/phim-moi-cap-nhap/trang-1.html'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
movie_items = soup.find_all('div', class_='movie-item')

In [17]:
def get_checkpoint_data(movie) -> tuple[int, str]:

    movie_id = int(movie['id'][9:])
    movie_latest_ep = movie.find('div', class_='episode-latest').text
    
    return (movie_id, movie_latest_ep)


def save_checkpoint(checkpoint_set):

    import pickle
    CHECKPOINT_PATH = './data/checkpoint.pkl'

    with open(CHECKPOINT_PATH, 'wb') as f:
        pickle.dump(checkpoint_set, f)


def load_checkpoint() -> set[int, str]:
    import pickle
    CHECKPOINT_PATH = './data/checkpoint.pkl'

    with open(CHECKPOINT_PATH, 'rb') as f:
        data = pickle.load(f)

    return data


def get_movie_items(page):

    url = f'https://animehay.in/phim-moi-cap-nhap/trang-{page}.html'
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    movie_items = soup.find_all('div', class_='movie-item')

    return movie_items



def extract_data(movie) -> tuple:
    
    from requests import  ConnectionError

    link = movie.select('a:nth-child(2)')[0]['href']
    response = requests.get(link)
    
    if response.status_code != 200:
        return ()

    soup = BeautifulSoup(response.content, 'html.parser')
    
    movie_data = soup.find('div', class_='last')

    if movie_data is None:
        return ()
    
    name = soup.find('h1', class_='heading_movie').text.strip()

    raw_categories = movie_data.find('div', class_='list_cate') \
                            .text.split('\n')[4:-1:2]
    categories = [category.strip() for category in raw_categories]

    movie_status = movie_data.find('div', class_='status') \
                    .select('div:nth-child(2)')[0] \
                    .text.strip()
    
    score_and_review = movie_data.find('div', class_='score') \
                                .select('div:nth-child(2)')[0].text.split()[:-2:2]
    score, review = score_and_review[0], score_and_review[1]

    publish_year = movie_data.find('div', class_='update_time') \
                            .select('div:nth-child(2)')[0].text.strip()
    
    duration = movie_data.find('div', class_='duration') \
                        .select('div:nth-child(2)')[0].text.strip()
    
    return (name, categories, movie_status, score, review, publish_year, duration, link)

def convert_to_dict(id, data) -> dict:

    result_dict = {
        'id': id,
        'name': data[0],
        'categories': data[1],
        'status': data[2],
        'score': float(data[3]),
        'review': int(data[4]),
        'publish_year': int(data[5]),
        'duration': data[6],
        'link': data[7]
    }
    return result_dict


# def save_to_csv(movie_data):

#     import pandas as pd
    

#     file_name = datetime.today().strftime('%Y_%m_%d')


def save_to_json(data):

    import json
    from datetime import datetime

    file_name = datetime.today().strftime('%Y_%m_%d')
    with open(f'./data/json/{file_name}.json', 'w', encoding='utf-8') as f:
        json.dump(data, f)


In [18]:
# Infinite scrapper
proceed = True
page = 14
new_checkpoint_data = set()
json_data = []
current_checkpoint_data = load_checkpoint()
# print(current_checkpoint_data)
while(proceed):

    print(page)
    movie_items = get_movie_items(page)
    for movie in movie_items:

        checkpoint_data = get_checkpoint_data(movie)

        # # Check if the movie item has ben processed on the previous batch or not
        # if checkpoint_data in current_checkpoint_data:
        #     # save_checkpoint(new_checkpoint_data)  # Save this batch as checkpoint for future scrapping batches
        #     proceed = False  # Stop this movie item processing batch
        #     break  

        new_checkpoint_data.add(checkpoint_data)

        data = extract_data(movie)
        if len(data) == 0:  # Skip to next movie in case there is no data
            continue

        json_data.append(convert_to_dict(id=checkpoint_data[0], data=data))
        print(data)

    page += 1
    if page > 16:
        proceed = False

save_to_json(json_data)

14
('Đan Đạo Tông Sư', ['CN Animation', 'Kiếm hiệp', 'Huyền ảo'], 'Đang tiến hành', '5.8', '122', '2023', '40 Tập', 'https://animehay.in/thong-tin-phim/dan-dao-tong-su-3719.html')
('Mobile Suit Gundam: Suisei no Majo Season 2', ['Anime', 'Hành động', 'Mecha', 'Viễn tưởng'], 'Hoàn thành', '7.0', '69', '2023', '12 Tập', 'https://animehay.in/thong-tin-phim/mobile-suit-gundam-suisei-no-majo-season-2-3776.html')
('Ở Rể', ['CN Animation'], 'Đang tiến hành', '8.7', '316', '2023', '12 Tập', 'https://animehay.in/thong-tin-phim/o-re-3781.html')
('Jigokuraku', ['Anime', 'Hành động', 'Giả tưởng', 'Lịch sử', 'Kinh dị'], 'Hoàn thành', '9.6', '604', '2023', '13 Tập', 'https://animehay.in/thong-tin-phim/jigokuraku-3727.html')
('Mashle', ['Anime', 'Hành động', 'Hài hước', 'Giả tưởng', 'Học đường', 'Shounen'], 'Hoàn thành', '9.3', '608', '2023', '12 Tập', 'https://animehay.in/thong-tin-phim/mashle-3752.html')
('Isekai de Cheat Skill wo Te ni Shita Ore wa, Genjitsu Sekai wo mo Musou Suru: Level Up wa Jin

In [26]:
import pickle
with open('./data/checkpoint.pkl', 'wb') as f:
    pickle.dump(set([(2713, '84/84')]), f)

In [28]:
with open('./data/checkpoint.pkl', 'rb') as f:
    data = pickle.load(f)

data

{(2713, '84/84')}

In [ ]:
page = 0



Quyến Tư Lượng ['CN Animation', 'Tiên hiệp', 'Huyền ảo'] Hoàn thành 9.2 375 2021 30 Tập
Na Nare Hana Nare ['Anime', 'Thể thao'] Đang tiến hành 5.4 14 2024 ?? Tập
Isekai Shikkaku ['Anime', 'Hành động', 'Hài hước', 'Phiêu lưu'] Đang tiến hành 8.6 62 2024 ?? Tập
Vạn Giới Độc Tôn ['CN Animation', 'Kiếm hiệp', 'Huyền ảo', 'Dị giới'] Đang tiến hành 8.1 549 2021 274 Tập
Nhất Niệm Vĩnh Hằng ['CN Animation', 'Tiên hiệp', 'Huyền ảo'] Đang tiến hành 9.0 2239 2020 135 Tập
Già Thiên ['CN Animation', 'Tiên hiệp', 'Xuyên không', 'Huyền ảo'] Đang tiến hành 8.6 1312 2023 104 Tập


In [35]:
pages = soup.find_all('div', class_='pagination')

page_links = [page.div for page in pages]

In [79]:
'https://animehay.in/thong-tin-phim/tien-vo-truyen-3723.html'

'https://animehay.in/thong-tin-phim/tien-vo-truyen-3723.html'

In [55]:
a_s = pages[0].find_all('a')

In [56]:
a_s

[<a href="https://animehay.in/phim-moi-cap-nhap/trang-1.html">Đầu</a>,
 <a class="active_page" href="https://animehay.in/phim-moi-cap-nhap/trang-1.html">1</a>,
 <a href="https://animehay.in/phim-moi-cap-nhap/trang-2.html">2</a>,
 <a href="javascript:toggleGoPage()" style="background: #a54f4f;">GO</a>,
 <a href="https://animehay.in/phim-moi-cap-nhap/trang-137.html">Cuối</a>]

In [102]:
len(movie_links)

30

# scrap -> check -> turn page or filter

In [ ]:
def get_movie_list